In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
dataset_path = '/kaggle/input/semantic-segmentation'

In [3]:
image_files, images, masks = [], [], []

for i in range(100):
    image_files.append(f"Frame 1  ({i+1}).jpg")

    # Load RGB image
    image_path = os.path.join(dataset_path, image_files[i])
    img = load_img(image_path, target_size=(224, 224))
    img_array = img_to_array(img) / 255.0  # Normalize pixels
    images.append(img_array)

    # Load corresponding mask
    mask_path = os.path.join(dataset_path, f"{image_files[i]}___fuse.png")
    mask = load_img(mask_path, target_size=(224, 224), color_mode='grayscale')
    mask_array = img_to_array(mask) / 255.0  # Normalize pixels
    masks.append(mask_array)

image_files

['Frame 1  (1).jpg',
 'Frame 1  (2).jpg',
 'Frame 1  (3).jpg',
 'Frame 1  (4).jpg',
 'Frame 1  (5).jpg',
 'Frame 1  (6).jpg',
 'Frame 1  (7).jpg',
 'Frame 1  (8).jpg',
 'Frame 1  (9).jpg',
 'Frame 1  (10).jpg',
 'Frame 1  (11).jpg',
 'Frame 1  (12).jpg',
 'Frame 1  (13).jpg',
 'Frame 1  (14).jpg',
 'Frame 1  (15).jpg',
 'Frame 1  (16).jpg',
 'Frame 1  (17).jpg',
 'Frame 1  (18).jpg',
 'Frame 1  (19).jpg',
 'Frame 1  (20).jpg',
 'Frame 1  (21).jpg',
 'Frame 1  (22).jpg',
 'Frame 1  (23).jpg',
 'Frame 1  (24).jpg',
 'Frame 1  (25).jpg',
 'Frame 1  (26).jpg',
 'Frame 1  (27).jpg',
 'Frame 1  (28).jpg',
 'Frame 1  (29).jpg',
 'Frame 1  (30).jpg',
 'Frame 1  (31).jpg',
 'Frame 1  (32).jpg',
 'Frame 1  (33).jpg',
 'Frame 1  (34).jpg',
 'Frame 1  (35).jpg',
 'Frame 1  (36).jpg',
 'Frame 1  (37).jpg',
 'Frame 1  (38).jpg',
 'Frame 1  (39).jpg',
 'Frame 1  (40).jpg',
 'Frame 1  (41).jpg',
 'Frame 1  (42).jpg',
 'Frame 1  (43).jpg',
 'Frame 1  (44).jpg',
 'Frame 1  (45).jpg',
 'Frame 1  (46).jpg

In [4]:
images = np.array(images)
masks = np.array(masks)

In [5]:
train_images, val_images, train_masks, val_masks = train_test_split(
    images, masks, test_size=0.2, random_state=42, shuffle=True
)

In [6]:
# Convert masks to one-hot encoding
train_masks = to_categorical(train_masks, num_classes=11)
val_masks = to_categorical(val_masks, num_classes=11)

In [7]:
# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

In [8]:
# U-Net model
model = Sequential(name='U-Net')

# Encoder
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Bottleneck
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))

# Decoder
model.add(UpSampling2D(size=(2, 2)))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))

model.add(UpSampling2D(size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))

model.add(UpSampling2D(size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))

# Output
model.add(Conv2D(11, (1, 1), activation='softmax'))

In [9]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
model.summary()

Model: "U-Net"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2  (None, 112, 112, 64)      0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 112, 112, 128)     73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 112, 112, 128)     147584    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 56, 56, 128)       0         
 g2D)                                                        

In [11]:
history = model.fit(
    datagen.flow(train_images, train_masks, batch_size=32),
    epochs=10,
    validation_data=(val_images, val_masks)
)

Epoch 1/10
3/3 [==============================] - 31s 5s/step - loss: 1.3761 - accuracy: 0.5960 - val_loss: 0.1392 - val_accuracy: 0.9954
Epoch 2/10
3/3 [==============================] - 2s 606ms/step - loss: 1.6196e-08 - accuracy: 1.0000 - val_loss: 44.6569 - val_accuracy: 0.9360
Epoch 3/10
3/3 [==============================] - 2s 393ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 39346.8828 - val_accuracy: 0.0045
Epoch 4/10
3/3 [==============================] - 2s 448ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 225349.7031 - val_accuracy: 7.9719e-05
Epoch 5/10
3/3 [==============================] - 2s 411ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 641325.3125 - val_accuracy: 1.9930e-05
Epoch 6/10
3/3 [==============================] - 2s 416ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 1292350.1250 - val_accuracy: 0.0000e+00
Epoch 7/10
3/3 [==============================] - 2s 606ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_l

In [12]:
test_loss, test_accuracy = model.evaluate(val_images, val_masks)

1/1 [==============================] - 0s 147ms/step - loss: 4794417.5000 - accuracy: 0.0000e+00
